<a href="https://colab.research.google.com/github/htapiagroup/sistemas-distribuidos-evaluacion-intersemestral-asosam91/blob/master/MCA_SistDist_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adrián Sosa Medellín
## MCA 2018-2020

### Configuracion inicial

In [0]:
import os, time, psutil
import pandas as pd 
import matplotlib as plt
import dask as dk 
import dask.dataframe as dd
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
cd gdrive/My\ Drive/Colab\ Notebooks/Data


/content/gdrive/My Drive/Colab Notebooks/Data


### definicion de funciones

In [0]:
# esta funcion fue proporcionada por el Dr. Horacio Tapia
def memoryPrint():
  '''Regresa al memoria usada por un proceso en python'''
  mem = psutil.Process(os.getpid()).memory_info().rss
  return (mem/1024**2)

### descarga y descompresion de la 'Base de datos'

In [0]:
if not os.path.exists('trip_data.7z'):
  !wget https://archive.org/download/nycTaxiTripData2013/trip_data.7z && !7z x trip_data.7z

### Parte 3, preguntas 1 y 2

Responder las siguientes preguntas:
> Cuantas columnas contiene cada archivo de datos descomprimido
> Cuantos renglones tiene cada archivo


In [0]:
ti = time.time()
m1 =  memoryPrint()
df = pd.read_csv('trip_data_1.csv')
m2 =  memoryPrint()
tf = time.time()

In [0]:
dimensiones = df.shape

In [8]:
print("El archivo contine {} columnas y {} renglones". format(dimensiones[1], dimensiones[0]))

El archivo contine 14 columnas y 14776615 renglones


### parte 4 preguntas 1 y 2
Identificar si existen renglones con errores en los datos, por ejemplo si hay columnas de mas (o de menos), si hay campos vacios, etc. Si se detectan renglones con errores:
> indicar claramente cuantos son los renglones con errores y a que archivos corresponden

> crear un nuevo conjunto de datos con los errores eliminados

In [9]:
df.isna().sum()

medallion                   0
hack_license                0
vendor_id                   0
rate_code                   0
store_and_fwd_flag    7326207
pickup_datetime             0
dropoff_datetime            0
passenger_count             0
trip_time_in_secs           0
trip_distance               0
pickup_longitude            0
pickup_latitude             0
dropoff_longitude          86
dropoff_latitude           86
dtype: int64

Existen 86 registros vacios en las columnas *dropoff_longitude* y *dropoff_latitude*.

en la columna *store_and_fwd_flag* hay 7326207 registros vacios


In [10]:
# aqui se borran los campos vacios 
df = df.dropna()
df.shape

(7450408, 14)

se genra un nuevo dataframe de 14 columnas y 7450408 registros.

se verifica que no haya mas registros vacios.

In [11]:
df.isna().sum()

medallion             0
hack_license          0
vendor_id             0
rate_code             0
store_and_fwd_flag    0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
trip_time_in_secs     0
trip_distance         0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
dtype: int64

### parte 5
Crear un DataFrame usando la libreria pandas y responder lo siguiente:
> Indicar el tiempo en segundos, que tarda la libreria pandas en leer un archivo a un DataFrame

> Cual es la huella de memoria del proceso usado para generar el objeto DataFrame

> Cuanta memoria del sistema se usa para crear el objeto anterior

> Indicar el tiempo que tarda pandas en obtener el promedio de la distancia de viaje (trip_distance)

In [12]:
print ("tiempo de lectura con pandas: {:.02f} s ".format(tf-ti))

tiempo de lectura con pandas: 55.27 s 


In [13]:
print("la huella de memoria utilizada por pandas al leer el archivo es de: {}".format(m2-m1))

la huella de memoria utilizada por pandas al leer el archivo es de: 4963.9375


In [14]:
print("el objeto anterior utiliza {} mb para ser creado".format(df.memory_usage().sum()//(1024**2)))

el objeto anterior utiliza 852 mb para ser creado


In [15]:
ti = time.time()


tf = time.time()
print ("tiempo de calculo de promedio con pandas: {:.02f} s ".format((((tf-ti)*1000)/60)/60))

tiempo de calculo de promedio con pandas: 0.00 s 
